In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
print(os.getcwd())

/opt/spark/work-dir/notebooks


In [3]:
import sys
from spark.spark_config import create_spark_session
spark = create_spark_session()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-08c29773-f983-4fb6-a918-cde1396ec948;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.5.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.5.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	fou

In [4]:
from data_loader import DataLoader
from preprocessor import Preprocessor
from nlp_processor import NLPProcessor
from feature_extractor import FeatureExtractor


In [5]:
data_loader = DataLoader(spark)
df = data_loader.load_sample()

In [6]:
# analyser = EdaAnalyser(df)
# analyser.run_full_eda_report(['title', 'lyrics', 'views'])


In [7]:
df = Preprocessor.run(df)


In [8]:
df_tokenized = NLPProcessor.run(df)
df_tokenized.cache()

DataFrame[title: string, tag: string, artist: string, year: int, views: int, features: string, lyrics: string, id: int, language_cld3: string, language_ft: string, language: string, lyrics_cleaned: string, words_lemmatized: array<string>]

In [19]:
extractor = FeatureExtractor(vocabSize=10000, minDF=5.0)
# extractor.fit(df_tokenized)
# df_final = extractor.transform(df_tokenized)
# df_tokenized.unpersist()

In [26]:
from pathlib import Path
base_dir = Path.cwd().parent
test_file_path = base_dir / "models" / "test" / "tf-idf"
# extractor.save(str(test_file_path))
df_final = extractor.load(str(test_file_path)).transform(df_tokenized)

In [27]:
df_final.show(3)

+--------------------+---+--------------------+----+-----+--------------------+--------------------+-----+-------------+-----------+--------+--------------------+--------------------+--------------------+
|               title|tag|              artist|year|views|            features|              lyrics|   id|language_cld3|language_ft|language|      lyrics_cleaned|    words_lemmatized|         tfidf_array|
+--------------------+---+--------------------+----+-----+--------------------+--------------------+-----+-------------+-----------+--------+--------------------+--------------------+--------------------+
|CJ KaZet - My wor...|rap|Polskie tumaczeni...|2011|   42|{"Polskie tłumacz...|(intro)\nSłuchaj,...|53105|           pl|         pl|      pl|Słuchaj, czy zast...|[słuchaj, zastana...|[0.0, 0.0, 0.0, 0...|
|               Gniew|rap|              Pi Dwa|2008| 4883|        {"Pięć Dwa"}|[Intro]\n- Czy Ty...|58000|           pl|         pl|      pl|- Czy Ty nie masz...|[mieć, żaden, ucz.

In [30]:
from repository.cassandra_provider import CassandraProvider 
cassandra_provider = CassandraProvider()

In [31]:
cassandra_provider.save(df_final)

Py4JJavaError: An error occurred while calling o505.save.
: java.lang.NoSuchMethodError: 'boolean org.apache.spark.sql.connector.write.streaming.StreamingWrite.useCommitCoordinator()'
	at com.datastax.spark.connector.datasource.CassandraBulkWrite.useCommitCoordinator(CassandraWriteBuilder.scala:116)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:397)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:382)
	at org.apache.spark.sql.execution.datasources.v2.AppendDataExec.writeWithV2(WriteToDataSourceV2Exec.scala:248)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run(WriteToDataSourceV2Exec.scala:360)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run$(WriteToDataSourceV2Exec.scala:359)
	at org.apache.spark.sql.execution.datasources.v2.AppendDataExec.run(WriteToDataSourceV2Exec.scala:248)
	at org.apache.spark.sql.

In [32]:
spark.version

'3.4.0'